# B02. Simulations
- Simulate matchups

In [1]:
%run "U1. Imports.ipynb"
# This needs to be run twice
# %run "U1. Imports.ipynb"
%run "U2. Utilities.ipynb"
%run "U3. Classes.ipynb"
# %run "D3. Simulation Functions.ipynb"

baseball_path = r'C:\Users\james\Documents\MLB\Database'

db_path = r'C:\Users\james\Documents\MLB\Database\MLBDB.db'
engine = create_engine(f'sqlite:///{db_path}')

In [ ]:
batter_stats_scaler

### Scale Inputs

In [ ]:
def scale_inputs(away_batter_df, away_pitcher_df, home_batter_df, home_pitcher_df, batter_stats_scaler, batter_stats_fg_scaler, pitcher_stats_scaler, pitcher_stats_fg_scaler):
    ### Standardize stats
    # Away batters
    away_batter_df[batter_stats_l] = batter_stats_scaler.fit_transform(away_batter_df[batter_stats_l])
    away_batter_df[batter_stats_r] = batter_stats_scaler.fit_transform(away_batter_df[batter_stats_r])
    away_batter_df[batter_stats_fg] = batter_stats_fg_scaler.fit_transform(away_batter_df[batter_stats_fg])
    # Away pitchers
    away_pitcher_df[pitcher_stats_l] = pitcher_stats_scaler.fit_transform(away_pitcher_df[pitcher_stats_l])
    away_pitcher_df[pitcher_stats_r] = pitcher_stats_scaler.fit_transform(away_pitcher_df[pitcher_stats_r])
    away_pitcher_df[pitcher_stats_fg2] = pitcher_stats_fg_scaler.fit_transform(away_pitcher_df[pitcher_stats_fg2])

    # Home batters
    home_batter_df[batter_stats_l] = batter_stats_scaler.fit_transform(home_batter_df[batter_stats_l])
    home_batter_df[batter_stats_r] = batter_stats_scaler.fit_transform(home_batter_df[batter_stats_r])
    home_batter_df[batter_stats_fg] = batter_stats_fg_scaler.fit_transform(home_batter_df[batter_stats_fg])
    # Home pitchers
    home_pitcher_df[pitcher_stats_l] = pitcher_stats_scaler.fit_transform(home_pitcher_df[pitcher_stats_l])
    home_pitcher_df[pitcher_stats_r] = pitcher_stats_scaler.fit_transform(home_pitcher_df[pitcher_stats_r])
    home_pitcher_df[pitcher_stats_fg2] = pitcher_stats_fg_scaler.fit_transform(home_pitcher_df[pitcher_stats_fg2])
    
    return away_batter_df, away_pitcher_df, home_batter_df, home_pitcher_df

### Impute Inputs

### Create Matchup Objects

In [ ]:
# Create team-position objects
def create_matchup(matchup, matchup_path, batter_stats_scaler, batter_stats_fg_scaler, pitcher_stats_scaler, pitcher_stats_fg_scaler):
    ### Read in data
    # Batters
    away_batter_df = pd.read_excel(os.path.join(matchup_path, matchup), sheet_name='AwayBatters', engine='openpyxl')
    home_batter_df = pd.read_excel(os.path.join(matchup_path, matchup), sheet_name='HomeBatters', engine='openpyxl')
    
    # Pitchers
    away_pitcher_df = pd.read_excel(os.path.join(matchup_path, matchup), sheet_name='AwayPitchers', engine='openpyxl')
    home_pitcher_df = pd.read_excel(os.path.join(matchup_path, matchup), sheet_name='HomePitchers', engine='openpyxl')

    ### Scale inputs
    away_batter_df, away_pitcher_df, home_batter_df, home_pitcher_df = scale_inputs(away_batter_df, away_pitcher_df, home_batter_df, home_pitcher_df, batter_stats_scaler, batter_stats_fg_scaler, pitcher_stats_scaler, pitcher_stats_fg_scaler)
    
    
    # Column names
    batter_columns = away_batter_df.columns.tolist()
    pitcher_columns = away_pitcher_df.columns.tolist()
   
    
    ### Create player objects
    # Lists of player objects, by Away/Home status and position group
    AwayBatters = []
    HomeBatters = []
    AwayPitchers = []
    HomePitchers = []
    
    # Away Batters
    for _, row in away_batter_df.iterrows():
        batter_data = {attr: row[attr] for attr in batter_columns}
        AwayBatters.append(Batter(**batter_data))
    
    # Home Batters
    for _, row in home_batter_df.iterrows():
        batter_data = {attr: row[attr] for attr in batter_columns}
        HomeBatters.append(Batter(**batter_data))

    # Away Pitchers
    for _, row in away_pitcher_df.iterrows():
        pitcher_data = {attr: row[attr] for attr in pitcher_columns}
        AwayPitchers.append(Pitcher(**pitcher_data))
    
    # Home Pitchers
    for _, row in home_pitcher_df.iterrows():
        pitcher_data = {attr: row[attr] for attr in pitcher_columns}
        HomePitchers.append(Pitcher(**pitcher_data))
    

    return AwayBatters, HomeBatters, AwayPitchers, HomePitchers 

### Calculate Pull Odds

In [ ]:
# Calculate odds of being pulled
def pull_odds(game, model_pulls):
    # Determine batting and fielding team scores for use as inputs
    if game.top_bot == "Top":        
        batter_score = game.away_score
        pitcher_score = game.home_score
    else:
        batter_score = game.home_score
        pitcher_score = game.away_score
    
    # Pull model inputs (pull_inputs3)
    X_list = [
        game.pitching.B1, game.pitching.B2, game.pitching.B3, game.pitching.HR, game.pitching.BB, game.pitching.HBP, 
        game.pitching.SO, game.pitching.ER, game.pitching.faced, pitcher_score, batter_score, game.pitching.IP_start
            ]
    
    # Reshape to match what models expect
    model_inputs = pd.Series(X_list).values.reshape(1,-1)

    # Predict and make into a list
    pull_list = model_pulls.predict_proba(model_inputs).tolist()
    
    # Choose the probability that pull = 1
    odds = pull_list[0][1] 
    
    del X_list, model_inputs, pull_list
    
    return odds

### Determine PA Matchup

In [ ]:
def choose_pa_matchup(game, model_pulls, opener_list):    
    ### Top of the inning
    if game.top_bot == "Top":
        # Shuffle list of pitchers so that when a pitcher is decided based on being the first in the list with a given leverage, that pitcher will vary
        random.shuffle(game.home_pitchers)
       
        ### Determine leverage
        # If the starter is still in the game
        if game.home_starter_pulled == False:
            # Pitcher is the starter
            game.home_pitcher_up = next(pitcher for pitcher in game.home_pitchers if pitcher.Leverage == 1)
            game.pitching = game.home_pitcher_up
            # Roll to see if they'll be pulled
            pull_prob = pull_odds(game, model_pulls)
            pull_roll = random.random()
            if pull_roll < pull_prob:
                game.home_starter_pulled = True 
    
            # If they're an opener and it's the third inning,
            if game.inning >= 3 and game.home_starter.Name in opener_list:
                # Pull them
                game.home_starter_pulled = True
            
        # If the home starter has been pulled
        if game.home_starter_pulled == True:
            # Set leverage to 2 (low) by default
            game.home_leverage = 2
            # Set leverage to 3 (medium) if it's late
            if (game.inning > game.innings - 3):
                game.home_leverage = 3
            # Set leverage to 4 (high) if it's a save situation
            if (4 > (game.home_score - game.away_score) >= 0) and game.inning == game.innings and game.home_starter_pulled == True:
                game.home_leverage = 4
        
        # Pick pitcher
        try:
            # Try to use method above
            game.home_pitcher_up = next(pitcher for pitcher in game.home_pitchers if pitcher.Leverage == game.home_leverage)
        except:
            # Might not have a pitcher at every leverage. In that case, choose low leverage
            game.home_pitcher_up = next(pitcher for pitcher in game.home_pitchers if pitcher.Leverage == 2)    
    
        # Determine matchup
        game.pitching = game.home_pitcher_up
        game.ab = next(batter for batter in game.away_batters if batter.batting_order == game.away_order)
      
    
    
    ### Bottom of the inning
    elif game.top_bot == "Bot":
        # Shuffle list of pitchers so that when a pitcher is decided based on being the first in the list with a given leverage, that pitcher will vary
        random.shuffle(game.away_pitchers)
       
        ### Determine leverage
        # If the starter is still in the game
        if game.away_starter_pulled == False:
            # Pitcher is the starter
            game.away_pitcher_up = next(pitcher for pitcher in game.away_pitchers if pitcher.Leverage == 1)
            game.pitching = game.away_pitcher_up
            # Roll to see if they'll be pulled
            pull_prob = pull_odds(game, model_pulls)
            pull_roll = random.random()
            if pull_roll < pull_prob:
                game.away_starter_pulled = True 
    
            # If they're an opener and it's the third inning,
            if game.inning >= 3 and game.away_starter.Name in opener_list:
                # Pull them
                game.away_starter_pulled = True
            
        # If the home starter has been pulled
        if game.away_starter_pulled == True:
            # Set leverage to 2 (low) by default
            game.away_leverage = 2
            # Set leverage to 3 (medium) if it's late
            if (game.inning > game.innings - 3):
                game.away_leverage = 3
            # Set leverage to 4 (high) if it's a save situation
            if (4 > (game.away_score - game.home_score) >= 0) and game.inning == game.innings and game.away_starter_pulled == True:
                game.away_leverage = 4
        
        # Pick pitcher
        try:
            # Try to use method above
            game.away_pitcher_up = next(pitcher for pitcher in game.away_pitchers if pitcher.Leverage == game.away_leverage)
        except:
            # Might not have a pitcher at every leverage. In that case, choose low leverage
            game.away_pitcher_up = next(pitcher for pitcher in game.away_pitchers if pitcher.Leverage == 2)    
    
        # Determine matchup
        game.pitching = game.away_pitcher_up
        game.ab = next(batter for batter in game.home_batters if batter.batting_order == game.home_order)
    
    return game

### Probabilities

In [ ]:
# Create PA event probability
def probabilities(game, model_binary, model_outs, model_safe):
    
    # Batter attributes
    # Set batter attributes specific to pitcher hand
    for stat in batter_stats + ['imp_b']:
        if game.pitching.pitchHand == "Left":
            attr_name = f'{stat}_l'
        else:
            attr_name = f'{stat}_r'

        # Get the new attribute value
        new_value = getattr(game.ab, attr_name)

        # Set the attribute to the new value
        setattr(game.ab, stat, new_value)
        
    # Pitcher attributes
    # Set pitcher attributes specific to batter hand
    for stat in pitcher_stats + ['imp_p']:
        if game.ab.batSide == "Left" or (game.ab.batSide == "Switch" and game.pitching.pitchHand == "Right"):
            attr_name = f'{stat}_l'
        else:
            attr_name = f'{stat}_r'

        # Get the new attribute value
        new_value = getattr(game.pitching, attr_name)

        # Set the attribute to the new value
        setattr(game.pitching, stat, new_value)
        
    
    
    # Make compatible with model
    if game.ab.batSide == "Left" or (game.ab.batSide == "Switch" and game.pitching.pitchHand == "Right"):
        game.ab.b_L = 1
    else:
        game.ab.b_L = 0
    
    if game.pitching.pitchHand == "Left":
        game.pitching.p_L = 1
    else:
        game.pitching.p_L = 0
        
    # On base dummies
    if game.on_1b is not None:
        game.onFirst = 1
    else:
        game.onFirst = 0
    if game.on_2b is not None:
        game.onSecond = 1
    else:
        game.onSecond = 0
    if game.on_3b is not None:
        game.onThird = 1
    else: 
        game.onThird = 0
        
    # Top of the inning dummy
    if game.top_bot == "Top":
        game.top = 1
    else:
        game.top = 0

    # SCore differentials
    if game.top == 1:
        game.score_diff = game.away_score - game.home_score
    else:
        game.score_diff = game.home_score - game.away_score
    
    # Year dummies
    game.year_2015 = 0
    game.year_2016 = 0
    game.year_2017 = 0
    game.year_2018 = 0
    game.year_2019 = 0
    game.year_2020 = 0
    game.year_2021 = 0
    game.year_2022 = 0
    game.year_2023 = 1

    
    
    ### Inputs
    # Batters
    batter_inputs =  [getattr(game.ab, stat) for stat in batter_stats]
    
    # Pitchers
    pitcher_inputs = [getattr(game.pitching, stat) for stat in pitcher_stats]
    
    # Venues
    for num in venues:  # Iterate through your venue values
        venue_attribute_name = f'venue_{num}'  # Create the attribute name
        if getattr(game.ab.venue_id, 'venue_id', None) == num:
            setattr(game, venue_attribute_name, 1)
        else:
            setattr(game, venue_attribute_name, 0)
    venue_inputs = [getattr(game, venue) for venue in venues]

    # Years 
    year_inputs = [getattr(game, year) for year in years]
    
    
    
    # Other 
    other_inputs = [game.pitching.p_L, game.ab.b_L, 
                    game.ab.x_vect, game.ab.y_vect, game.ab.temperature, 
                    game.onFirst, game.onSecond, game.onThird, game.inning, game.top, game.score_diff]
    
    imp_inputs = [game.ab.imp_b, game.pitching.imp_p]
    
    ### Create inputs 
    inputs = batter_inputs + pitcher_inputs + venue_inputs + year_inputs + other_inputs + imp_inputs
    model_inputs = pd.Series(inputs).values.reshape(1,-1)
    
    ### Run models
    # Out or save
    binary_list = model_binary.predict_proba(model_inputs).tolist()
    # Out types
    outs_list = model_outs.predict_proba(model_inputs).tolist()
    # Safe types
    safe_list = model_safe.predict_proba(model_inputs).tolist()
    
    # Odds of safe/out
    is_safe = binary_list[0][0]
    is_out = 1 - is_safe
    
    # Probabilities 
    # Given that it's an out
    fo_pred = outs_list[0][0]
    go_pred = outs_list[0][1]
    lo_pred = outs_list[0][2]
    po_pred = outs_list[0][3]
    so_pred = outs_list[0][4]
    
    # Given that it's safe
    b1_pred = safe_list[0][0]
    b2_pred = safe_list[0][1]
    b3_pred = safe_list[0][2]
    bb_pred = safe_list[0][3]
    hbp_pred = safe_list[0][4]
    hr_pred = safe_list[0][5]
       
    # Overall probabilities
    b1 = is_safe * b1_pred
    b2 = is_safe * b2_pred
    b3 = is_safe * b3_pred
    bb = is_safe * bb_pred
    fo = is_out * fo_pred
    go = is_out * go_pred
    hbp = is_safe * hbp_pred
    hr = is_safe * hr_pred
    lo = is_out * lo_pred
    po = is_out * po_pred
    so = is_out * so_pred 
           
    # Add stats together so they represent the end point of a range with a probability of being selected equal to their rate
    bb = hbp + bb 
    b1 = bb + b1
    b2 = b1 + b2
    b3 = b2 + b3
    hr = b3 + hr
    so = hr + so
    lo = so + lo
    po = lo + po
    go = po + go
    fo = go + fo
    # Set flyout to be 1. This model almost guarantees this. It's accurate to like, 10 decimal places, but why risk it?
    fo = 1
    
    del model_inputs
    
    return hbp, bb, b1, b2, b3, hr, so, lo, po, go, fo

### Events

In [ ]:
# Hit by pitch
def event_hbp(game):
    # Runs
    runs = 0
    
    # Players
    game.pitching.HBP += 1
    game.ab.HBP += 1 
    
    # Bases: 1st, 2nd, 3rd
    if game.on_3b is not None and game.on_2b is not None and game.on_1b is not None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = game.on_2b
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
    # Bases: 1st, 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is not None:
        game.on_3b = game.on_2b
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
    # Bases: 1st, 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is not None:
        game.on_3b = game.on_3b
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
    # Bases: 2nd, 3rd
    elif game.on_3b is not None and game.on_2b is not None and game.on_1b is None:
        game.on_3b = game.on_3b
        game.on_2b = game.on_2b
        game.on_1b = game.ab
    
    # Bases: 1st
    elif game.on_3b is None and game.on_2b is None and game.on_1b is not None:
        game.on_3b = None
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
    # Bases: 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is None:
        game.on_3b = None
        game.on_2b = game.on_2b
        game.on_1b = game.ab
    
    # Bases: 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is None:
        game.on_3b = game.on_3b
        game.on_2b = None
        game.on_1b = game.ab
    
    # Bases: None
    elif game.on_3b is None and game.on_2b is None and game.on_1b is None:
        game.on_3b = None
        game.on_2b = None
        game.on_1b = game.ab
    
    # Runs scored
    if game.top_bot == "Top":
        game.away_score += runs
    else:
        game.home_score += runs
                
    return game

In [ ]:
# Walk
def event_bb(game):
    # Runs
    runs = 0
    
    # Players
    game.pitching.BB += 1
    game.ab.BB += 1 
    
    # Bases: 1st, 2nd, 3rd
    if game.on_3b is not None and game.on_2b is not None and game.on_1b is not None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = game.on_2b
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
    # Bases: 1st, 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is not None:
        game.on_3b = game.on_2b
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
    # Bases: 1st, 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is not None:
        game.on_3b = game.on_3b
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
    # Bases: 2nd, 3rd
    elif game.on_3b is not None and game.on_2b is not None and game.on_1b is None:
        game.on_3b = game.on_3b
        game.on_2b = game.on_2b
        game.on_1b = game.ab
    
    # Bases: 1st
    elif game.on_3b is None and game.on_2b is None and game.on_1b is not None:
        game.on_3b = None
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
    # Bases: 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is None:
        game.on_3b = None
        game.on_2b = game.on_2b
        game.on_1b = game.ab
    
    # Bases: 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is None:
        game.on_3b = game.on_3b
        game.on_2b = None
        game.on_1b = game.ab
    
    # Bases: None
    elif game.on_3b is None and game.on_2b is None and game.on_1b is None:
        game.on_3b = None
        game.on_2b = None
        game.on_1b = game.ab
    
    # Runs scored
    if game.top_bot == "Top":
        game.away_score += runs
    else:
        game.home_score += runs
                
    return game

In [ ]:
# Single
def event_1b(game):
    # Runs
    runs = 0
    
    # Players
    game.pitching.B1 += 1
    game.ab.B1 += 1 
    
    # Bases: 1st, 2nd, 3rd
    if game.on_3b is not None and game.on_2b is not None and game.on_1b is not None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = game.on_2b
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
        # Do they try to score?
        extra_base_roll = random.random()
        if extra_base_roll < 0.601217:                
            game.on_3b.R += 1
            game.pitching.ER += 1
            game.ab.RBI += 1
            runs += 1
            
            game.on_3b = None
        
    # Bases: 1st, 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is not None:
        game.on_3b = game.on_2b
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
        # Do they try to score?
        extra_base_roll = random.random()
        if extra_base_roll < 0.601217:                
            game.on_3b.R += 1
            game.pitching.ER += 1
            game.ab.RBI += 1
            runs += 1
            
            game.on_3b = None
        
    # Bases: 1st, 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is not None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1        
        
        game.on_3b = None
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
    # Bases: 2nd, 3rd
    elif game.on_3b is not None and game.on_2b is not None and game.on_1b is None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1     
        
        game.on_3b = game.on_2b
        game.on_2b = None 
        game.on_1b = game.ab
        
        # Do they try to score?
        extra_base_roll = random.random()
        if extra_base_roll < 0.601217:                
            game.on_3b.R += 1
            game.pitching.ER += 1
            game.ab.RBI += 1
            runs += 1
            
            game.on_3b = None
    
    # Bases: 1st
    elif game.on_3b is None and game.on_2b is None and game.on_1b is not None:
        game.on_3b = None
        game.on_2b = game.on_1b
        game.on_1b = game.ab
        
    # Bases: 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is None:
        game.on_3b = game.on_2b
        game.on_2b = None
        game.on_1b = game.ab
        
        # Do they try to score?
        extra_base_roll = random.random()
        if extra_base_roll < 0.601217:                
            game.on_3b.R += 1
            game.pitching.ER += 1
            game.ab.RBI += 1
            runs += 1
            
            game.on_3b = None
    
    # Bases: 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = None
        game.on_2b = None
        game.on_1b = game.ab
    
    # Bases: None
    elif game.on_3b is None and game.on_2b is None and game.on_1b is None:
        game.on_3b = None
        game.on_2b = None
        game.on_1b = game.ab
    
    # Runs scored
    if game.top_bot == "Top":
        game.away_score += runs
    else:
        game.home_score += runs
                
    return game

In [ ]:
# Double
def event_2b(game):
    # Runs
    runs = 0
    
    # Players
    game.pitching.B2 += 1
    game.ab.B2 += 1 
    
    # Bases: 1st, 2nd, 3rd
    if game.on_3b is not None and game.on_2b is not None and game.on_1b is not None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = game.on_1b
        game.on_2b = game.ab
        game.on_1b = None
        
        # Do they try to score?
        extra_base_roll = random.random()
        if extra_base_roll < 0.396065:                
            game.on_3b.R += 1
            game.pitching.ER += 1
            game.ab.RBI += 1
            runs += 1
            
            game.on_3b = None
        
    # Bases: 1st, 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is not None:
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1        
        
        game.on_3b = game.on_1b
        game.on_2b = game.ab
        game.on_1b = None
        
        # Do they try to score?
        extra_base_roll = random.random()
        if extra_base_roll < 0.396065:                
            game.on_3b.R += 1
            game.pitching.ER += 1
            game.ab.RBI += 1
            runs += 1
            
            game.on_3b = None
        
    # Bases: 1st, 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is not None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1        
        
        game.on_3b = game.on_1b
        game.on_2b = game.ab
        game.on_1b = None
        
        # Do they try to score?
        extra_base_roll = random.random()
        if extra_base_roll < 0.396065:                
            game.on_3b.R += 1
            game.pitching.ER += 1
            game.ab.RBI += 1
            runs += 1
            
            game.on_3b = None
        
    # Bases: 2nd, 3rd
    elif game.on_3b is not None and game.on_2b is not None and game.on_1b is None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = None
        game.on_2b = game.ab
        game.on_1b = None
        
    # Bases: 1st
    elif game.on_3b is None and game.on_2b is None and game.on_1b is not None:
        game.on_3b = game.on_1b
        game.on_2b = game.ab
        game.on_1b = None
        
        # Do they try to score?
        extra_base_roll = random.random()
        if extra_base_roll < 0.396065:                
            game.on_3b.R += 1
            game.pitching.ER += 1
            game.ab.RBI += 1
            runs += 1
            
            game.on_3b = None
        
    # Bases: 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is None:
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = None
        game.on_2b = game.ab
        game.on_1b = None
    
    # Bases: 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = None
        game.on_2b = game.ab 
        game.on_1b = None
    
    # Bases: None
    elif game.on_3b is None and game.on_2b is None and game.on_1b is None:
        game.on_3b = None
        game.on_2b = game.ab
        game.on_1b = None
    
    # Runs scored
    if game.top_bot == "Top":
        game.away_score += runs
    else:
        game.home_score += runs
                
    return game

In [ ]:
# Triple
def event_3b(game):
    # Runs
    runs = 0
    
    # Players
    game.pitching.B3 += 1
    game.ab.B3 += 1 
    
    # Bases: 1st, 2nd, 3rd
    if game.on_3b is not None and game.on_2b is not None and game.on_1b is not None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_1b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = game.ab
        game.on_2b = None
        game.on_1b = None
        
    # Bases: 1st, 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is not None:
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1    
        
        game.on_1b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1    
        
        game.on_3b = game.ab
        game.on_2b = None
        game.on_1b = None
        
    # Bases: 1st, 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is not None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_1b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1    
        
        game.on_3b = game.ab
        game.on_2b = None
        game.on_1b = None
        
    # Bases: 2nd, 3rd
    elif game.on_3b is not None and game.on_2b is not None and game.on_1b is None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = game.ab
        game.on_2b = None
        game.on_1b = None
        
    # Bases: 1st
    elif game.on_3b is None and game.on_2b is None and game.on_1b is not None:
        game.on_1b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = game.ab
        game.on_2b = None
        game.on_1b = None        
        
    # Bases: 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is None:
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = game.ab
        game.on_2b = None
        game.on_1b = None   
    
    # Bases: 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = game.ab
        game.on_2b = None
        game.on_1b = None   
    
    # Bases: None
    elif game.on_3b is None and game.on_2b is None and game.on_1b is None:
        game.on_3b = game.ab
        game.on_2b = None
        game.on_1b = None   
    
    # Runs scored
    if game.top_bot == "Top":
        game.away_score += runs
    else:
        game.home_score += runs
                
    return game

In [ ]:
# Home Run
def event_hr(game):
    # Runs
    runs = 0
    
    # Players
    game.pitching.HR += 1
    game.ab.HR += 1 
    
    # Bases: 1st, 2nd, 3rd
    if game.on_3b is not None and game.on_2b is not None and game.on_1b is not None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_1b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.ab.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = None
        game.on_2b = None
        game.on_1b = None
        
    # Bases: 1st, 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is not None:
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1    
        
        game.on_1b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1   
        
        game.ab.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1 
        
        game.on_3b = None
        game.on_2b = None
        game.on_1b = None
        
    # Bases: 1st, 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is not None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_1b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1    
        
        game.ab.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = None
        game.on_2b = None
        game.on_1b = None
        
    # Bases: 2nd, 3rd
    elif game.on_3b is not None and game.on_2b is not None and game.on_1b is None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.ab.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = None
        game.on_2b = None
        game.on_1b = None
        
    # Bases: 1st
    elif game.on_3b is None and game.on_2b is None and game.on_1b is not None:
        game.on_1b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.ab.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = None
        game.on_2b = None
        game.on_1b = None        
        
    # Bases: 2nd
    elif game.on_3b is None and game.on_2b is not None and game.on_1b is None:
        game.on_2b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.ab.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = None
        game.on_2b = None
        game.on_1b = None   
    
    # Bases: 3rd
    elif game.on_3b is not None and game.on_2b is None and game.on_1b is None:
        game.on_3b.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.ab.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
        
        game.on_3b = None
        game.on_2b = None
        game.on_1b = None   
    
    # Bases: None
    elif game.on_3b is None and game.on_2b is None and game.on_1b is None:
        game.on_3b = None
        game.on_2b = None
        game.on_1b = None   
        
        game.ab.R += 1
        game.pitching.ER += 1
        game.ab.RBI += 1
        runs += 1
    
    # Runs scored
    if game.top_bot == "Top":
        game.away_score += runs
    else:
        game.home_score += runs
                
    return game

In [ ]:
# In play out
def event_ipo(game, out, dp_rates, dp_base_rates, advances):
    # Runs
    runs = 0
    
    game.pitching.OUT += 1
    game.outs += 1
    
    # Create flags for bases occupied. Use to index
    if game.on_1b is not None:
        runner_1b = 1
    else:
        runner_1b = 0
    if game.on_2b is not None:
        runner_2b = 1
    else:
        runner_2b = 0
    if game.on_3b is not None:
        runner_3b = 1
    else: 
        runner_3b = 0

    # Type of double play it would be
    out_dp = out + "_dp"
    
    # Odds that it's a double play given that it's that type of out
    dp_odds = dp_rates[out_dp].loc[(runner_1b, runner_2b, runner_3b)]

    
    # Double play odds are not conditional on there being an out. Make 'em so.
    if out == "ld":
        dp_odds = dp_odds / game.ab.lo_b
    elif out == "pu":
        dp_odds = dp_odds / game.ab.po_b
    elif out == "go":
        dp_odds = dp_odds / game.ab.go_b
    elif out == "fo":
        dp_odds = dp_odds / game.ab.fo_b
    
    # Is it a double play?
    dp_roll = random.random() 
    if dp_roll < dp_odds:
        # print("double play!")
        game.outs += 1
        game.pitching.OUT += 1
        
        # Second out 
        # At 1B
        dp_1b = out_dp + "_1b"
        dp_1b_odds = dp_base_rates[dp_1b].loc[(runner_1b, runner_2b, runner_3b)]
        
        # Ground outs are weird.
        if out == "go":
            dp_1b_odds = 0
        
        # At 2B
        dp_2b = out_dp + "_2b"
        dp_2b_odds = dp_base_rates[dp_2b].loc[(runner_1b, runner_2b, runner_3b)]
        
        # At 3B
        dp_3b = out_dp + "_3b"
        dp_3b_odds = dp_base_rates[dp_3b].loc[(runner_1b, runner_2b, runner_3b)] + dp_2b_odds
        
        # At home
        dp_h = out_dp + "_h"
        dp_h_odds = dp_base_rates[dp_h].loc[(runner_1b, runner_2b, runner_3b)] + dp_3b_odds
        
        # Two on the bases (only matters for GOs)
        dp_b = out_dp + "_b"
        dp_b_odds = dp_base_rates[dp_b].loc[(runner_1b, runner_2b, runner_3b)] + dp_h_odds
        
        # If the out is NOT a ground out!
        if out != "go":
            # Normalize by multiplying by max because sometimes it gets a little off
            dp_base_roll = random.random() * dp_b_odds
            # If it's an out at first
            if dp_base_roll < dp_1b_odds:
                # And there's nobody at first
                if game.on_1b is None:
                    # It's an out at second
                    game.on_2b = None
                # Else, it's just an out at first
                game.on_1b = None
            # If it's an out at second
            elif dp_base_roll < dp_2b_odds:
                # And there's nobody on second
                if game.on_2b is None:
                    # It's an out at first
                    game.on_1b = None
                game.on_2b = None
            # If it's an out at third
            elif dp_base_roll < dp_3b_odds:
                # And there's nobody on third or second
                if game.on_3b is None and game.on_2b is None:
                    # It's an out at first
                    game.on_1b = None
                # If there's nobody on third but there is on second 
                elif game.on_3b is None: 
                    # It's an out at second
                    game.on_2b = None
                game.on_3b = None   
            # If it's an out at home
            elif dp_base_roll < dp_h_odds:
                # It's an out at third
                game.on_3b = None
            # Can't be two outs on bases
            
        # If the out is a ground out
        if out == "go": 
            # Normalize by multiplying by max because sometimes it gets a little off
            dp_base_roll = random.random() * dp_b_odds
            # These are shifted to affect the runner 
            if dp_base_roll < dp_2b_odds:
                if game.on_1b is None:
                    game.on_2b = None
                game.on_1b = None
            elif dp_base_roll < dp_3b_odds:
                if game.on_2b is None and game.on_3b is None:
                    game.on_1b = None
                elif game.on_2b is None:
                    game.on_3b = None
                game.on_2b = None    
            elif dp_base_roll < dp_h_odds:
                game.on_3b = None 
            # There are two outs on the bases
            else:
                # If there are runners on second and third
                if game.on_1b is None and game.on_2b is not None and game.on_3b is not None:
                    # There are outs at second and third
                    game.on_2b = None
                    game.on_3b = None
                # If there are runners on first and third
                elif game.on_1b is not None and game.on_2b is None and game.on_3b is not None:
                    # There are outs at first and third
                    game.on_1b = None
                    game.on_3b = None
                # If there are runners on first and second
                elif game.on_1b is not None and game.on_2b is not None and game.on_3b is None:
                    # There are outs at first and second
                    game.on_1b = None
                    game.on_2b = None
                # If there are bases loaded
                elif game.on_1b is not None and game.on_2b is not None and game.on_3b is not None:
                    # There are outs at second and third (this is a simplifying assumption)
                    game.on_2b = None
                    game.on_3b = None
                # Batter reaches                
                game.on_1b = game.ab       
        dp = 1
        
    else:
        dp = 0
    
    # Advances 
    if game.outs < 3:
        if game.on_3b is not None:
            # Roll to see if they'll score
            advances_h_roll = random.random()
            advance_type = out + "_3b_h"
            if advances_h_roll < advances[advance_type][dp]: 
                game.on_3b.R += 1
                game.pitching.ER += 1
                game.on_3b = None
                if dp == 0:
                    game.ab.RBI += 1
                runs += 1
#                 print("Home")
                    
        if game.on_2b is not None and game.on_3b is None:
            # Roll to see if they'll go to third
            advances_3b_roll = random.random()
            advance_type = out + "_2b_3b"
            if advances_3b_roll < advances[advance_type][dp]: 
                game.on_3b = game.on_2b 
                game.on_2b = None
#                 print("Third")
        
                    
        if game.on_1b is not None and game.on_2b is None:
            # Roll to see if they'll go to second
            advances_2b_roll = random.random()
            advance_type = out + "_1b_2b"
            if advances_2b_roll < advances[advance_type][dp]: 
                game.on_2b = game.on_1b 
                game.on_1b = None
#                 print("Second")
            
    # Runs score
    if game.top_bot == "Top":
        game.away_score += runs
    else:
        game.home_score += runs

    return game    

In [ ]:
# Simulate at bat
def sim_ab(game, model_binary, model_outs, model_safe, model_pulls, opener_list):
    # Choose plate appearance matchup
    game = choose_pa_matchup(game, model_pulls, opener_list)
    # Calculate probabilities
    hbp, bb, b1, b2, b3, hr, so, lo, po, go, fo = probabilities(game, model_binary, model_outs, model_safe)
    
    # Roll
    pa_roll = random.random() 
    
    # Event 1: HBP
    if pa_roll < hbp:
        event = "hbp"
        game = event_hbp(game)
    # Event 2: BB
    elif pa_roll < bb:
        event = "bb"
        game = event_bb(game)
    # Event 3: Single
    elif pa_roll < b1:
        event = "1b"
        game = event_1b(game)
    # Event 4: Double    
    elif pa_roll < b2:
        event = "2b"
        game = event_2b(game)
    # Event 5: Triple
    elif pa_roll < b3:
        event = "3b"
        game = event_3b(game)
     # Event 6: Home Run
    elif pa_roll < hr:
        event = "hr"
        game = event_hr(game)
    # Event 7: Strikeout
    elif pa_roll < so:
        event = "so"
        game.outs += 1
        game.pitching.SO += 1
        game.pitching.OUT += 1
    # Event 8: Line drive out
    elif pa_roll < lo:
        event = "ld"
        game = event_ipo(game, event, dp_rates, dp_base_rates, advances)
    # Event 9: Pop out
    elif pa_roll < po:
        event = "pu"
        game = event_ipo(game, event, dp_rates, dp_base_rates, advances)
    # Event 10: Groundball out
    elif pa_roll < go:
        event = "go"
        game = event_ipo(game, event, dp_rates, dp_base_rates, advances)
    # Event 11: Fly out 
    else:
        event = "fo"
        game = event_ipo(game, event, dp_rates, dp_base_rates, advances)
    
    # Go to the next batter up
    if game.top_bot == "Top":
        game.away_order += 1
        if game.away_order == 10:
            game.away_order = 1

    else:
        game.home_order += 1
        if game.home_order == 10:
            game.home_order = 1
    
    print(event)
        
    return game

In [ ]:
def sim_inning(game, model_pulls, model_binary, model_outs, model_safe, dp_rates, dp_base_rates, advances, opener_list, innings=9):
    # Set outs to zero
    game.outs = 0
    # Clear bases
    game.on_1b = None
    game.on_2b = None
    game.on_2b = None
    game.onFirst = 0
    game.onSecond = 0
    game.onThird = 0
    
 
    # Loop for each PA
    while game.outs < 3:        
        ### Steals
        # Third base
        # If third is empty and second is not
        if game.on_3b is None and game.on_2b is not None:
            sba_3b_roll = random.random()
            # Attempt to steal third
            if sba_3b_roll < game.on_2b.sba_3b :
                sb_3b_roll = random.random()
                # They steal third
                if sb_3b_roll < game.on_2b.sb_3b:
                    game.on_2b.SB += 1
                    game.on_3b = game.on_2b
                    game.on_2b = None
                    
                # They're out
                else:
                    game.on_2b = None
                    game.outs += 1 
                    
        # Second base
        # If second is empty and first is not
        if game.on_2b is None and game.on_1b is not None:
            sba_2b_roll = random.random()
            # Attempt to steal second
            if sba_2b_roll < game.on_1b.sba_2b :
                sb_2b_roll = random.random()
                # They steal second
                if sb_2b_roll < game.on_1b.sb_2b:
                    game.on_1b.SB += 1
                    game.on_2b = game.on_1b
                    game.on_1b = None
                    
                # They're out
                else:
                    game.on_1b = None
                    game.outs += 1
                    
        
        ### Who gets the win?
        # If the away team is winning
        if game.away_score > game.home_score:
            # And it's the fifth inning or later (pitchers finish their innings, so this is fine)
            if game.inning >= 5:
                # And the current winning pitcher is not already on the away team (So if there is no current winning pitcher or he's on the other team)
                if game.winning_pitcher not in game.away_pitchers:
                    # The current pitcher is the winning pitcher
                    game.winning_pitcher = game.away_pitcher_up
        elif game.home_score > game.away_score:
            if game.inning >= 5:
                if game.winning_pitcher not in game.home_pitchers:
                    game.winning_pitcher = game.home_pitcher_up
        else:
            game.winning_pitcher = None
            
        # Simulate the AB
        game = sim_ab(game, model_binary, model_outs, model_safe, model_pulls, opener_list)
   
    return game

In [ ]:
# Calculate batter fantasy points
def calculate_batter(batter, game):
    batter.FP = (
                batter.B1 * 3 +
                batter.B2 * 5 +
                batter.B3 * 8 +
                batter.HR * 10 +
                batter.RBI * 2 +
                batter.R * 2 +
                batter.BB * 2 +
                batter.HBP * 2 +
                batter.SB * 5
                )
    
    return batter, game

In [ ]:
# Calculate pitcher fantasy points
def calculate_pitcher(pitcher, game):
    # Calculate hits allowed
    pitcher.H = (pitcher.B1 + pitcher.B2 + pitcher.B3 + pitcher.HR)

    # If they're the winning pitcher, they get a win
    if game.winning_pitcher == pitcher and game.inning >= game.innings:
        pitcher.W = 1
    else:
        pitcher.W = 0

    # Determine CG, CGSO, NH
    if pitcher.OUT == 27:
        pitcher.CG = 1
        if pitcher.ER == 0:
            pitcher.CGSO = 1
        if pitcher.H == 0:
            pitcher.NH = 1

    pitcher.FP = (
                pitcher.OUT * 0.75 +
                pitcher.SO * 2 +
                pitcher.W * 4 +
                pitcher.ER * -2 +
                pitcher.H * -0.6 +
                pitcher.BB * -0.6 +
                pitcher.HBP * -0.6 +
                pitcher.CG * 2.5 +
                pitcher.CGSO * 2.5 +
                pitcher.NH * 5
                )
    
    pitcher.PA = pitcher.OUT + pitcher.H + pitcher.BB + pitcher.HBP
    
    
    return pitcher, game

In [ ]:
# Simulate a whole game
def sim_game(game, pull, model_binary, model_outs, model_safe, dp_rates, dp_base_rates, advances, opener_list):    
    # Determine starters
    game.home_starter = next(pitcher for pitcher in game.home_pitchers if pitcher.Leverage == 1)
    game.away_starter = next(pitcher for pitcher in game.away_pitchers if pitcher.Leverage == 1)
    
    # Loop over every inning
    while game.inning <= game.innings:
        print(game.inning)
        # Calculate a pitcher's cumulative stats
        game.home_starter, game = calculate_pitcher(game.home_starter, game)
        game.away_starter, game = calculate_pitcher(game.away_starter, game)
        
        # Simulate the half inning  
        game = sim_inning(game, pull, model_binary, model_outs, model_safe, dp_rates, dp_base_rates, advances, opener_list, innings)

        # Switch halves
        if game.top_bot == "Top":
            game.top_bot = "Bot
        else:
            game.top_bot = "Top"
            game.inning = game.inning + 1 
        
        
        # If it's the last inning
        if (game.inning == game.innings):
            
        
        
        # If it's the bottom of the 9th and the home team is winning
        
        if (game.top_bot == "Bot") and (game.inning >= game.innings) and (game.home_score > game.away_score):
            # print("Game Over")
            
            for batter in game.home_batters:
                batter, game = calculate_batter(batter, game)
            for batter in game.away_batters:
                batter, game = calculate_batter(batter, game)
            for pitcher in game.home_pitchers:
                pitcher, game = calculate_pitcher(pitcher, game)
            for pitcher in game.away_pitchers:
                pitcher, game = calculate_pitcher(pitcher, game)

            return game

        # Else, simulate another half inning
        else:
            game = sim_inning(game, pull, model_binary, model_outs, model_safe, dp_rates, dp_base_rates, advances, opener_list, innings)
        
        if game.away_score == game.home_score and game.inning == innings + 1 and game.top_bot == "Top":
            # print("TIE")
            innings += 1         
        
        
    for batter in game.home_batters:
        batter, game = calculate_batter(batter, game)
    for batter in game.away_batters:
        batter, game = calculate_batter(batter, game)
    for pitcher in game.home_pitchers:
        pitcher, game = calculate_pitcher(pitcher, game)
    for pitcher in game.away_pitchers:
        pitcher, game = calculate_pitcher(pitcher, game)

    # Doesn't currently pick winning pitchers of walkoffs for some reason, probably not a big deal, but fix
    return game

In [ ]:
matchup = "WAS@LAA 04112023 0938PM ET.xlsx"
matchup_path = r"C:\Users\james\Documents\MLB\Database\B01. Matchups\Matchups 85143"

AwayBatters, HomeBatters, AwayPitchers, HomePitchers = create_matchup(matchup, matchup_path, batter_stats_scaler, batter_stats_fg_scaler, pitcher_stats_scaler, pitcher_stats_fg_scaler)

game = Scoreboard(AwayBatters, HomeBatters, AwayPitchers, HomePitchers, 9)
opener_list = []

In [ ]:
game = sim_inning(game, model_pulls, model_binary, model_outs, model_safe, dp_rates, dp_base_rates, advances, opener_list, innings=9)
